## Optical Character Recognition (OCR)

                            OCR
                             |
                      ------- ---------
                      |               | 
                Online Text      Offline Text
                                      |
                               ------- ---------
                               |               | 
                            Typed Text      Handwritten Text

In [1]:
import asrtoolkit
from asrtoolkit import wer, cer
import cv2
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import Levenshtein
import numpy as np
from PIL import Image
import pytesseract
import sys
#skimage causing issues with io standard library, hence import as
from skimage import io as ioski
from skimage.color import rgb2gray
from skimage.transform import rotate, resize
import os
from google.cloud import vision_v1
from google.cloud.vision_v1 import types
from dotenv import load_dotenv, find_dotenv

load_dotenv()


False

In [13]:
import os
from google.cloud import vision

# Assuming GOOGLE_CREDS is a string containing JSON credentials
google_creds = "cc2478d128e3122ba7b6d903ce7a01c560bf99a0" 

if google_creds:
    # Writing the credentials to a file
    credentials_path = "C:\\Users\\hicham\\Desktop\\Ai project\\tmp\\google.json"
    with open(credentials_path, "w") as fp:
        fp.write(google_creds)

    # Setting the environment variable for the credentials
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

    # Creating the ImageAnnotatorClient
    client = vision.ImageAnnotatorClient()
else:
    print("Error: GOOGLE_CREDS environment variable not set.")

DefaultCredentialsError: ('File C:\\Users\\hicham\\Desktop\\Ai project\\tmp\\google.json is not a valid json file.', JSONDecodeError('Expecting value: line 1 column 1 (char 0)'))

## Files paths

In [3]:
dirpath = os.getcwd() #os abosulute current directory path
print("This Directory",dirpath)
parentDirectory = os.path.dirname(dirpath)
print(parentDirectory)
GrandparentDirectory = os.path.dirname(parentDirectory)
print(GrandparentDirectory)
# bsolutepath = os.path.abspath(__file__) #os abosulute current file path
# print(bsolutepath)
print(os.path.abspath(os.path.join(os.getcwd(), '..')))
out_path_myhand = os.getenv("LOCAL_PATH")
print("My_hand_Trained_data",out_path_myhand)
out_path_StorySquad = os.path.abspath(os.path.join(os.getenv("LOCAL_PATH"), "..",".."))
print(out_path_StorySquad)

This Directory C:\Users\hicham\Desktop\Ai project
C:\Users\hicham\Desktop
C:\Users\hicham
C:\Users\hicham\Desktop
My_hand_Trained_data None


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [4]:
sys.path.insert(0, out_path_StorySquad+'/deskew')
from deskew import determine_skew

NameError: name 'out_path_StorySquad' is not defined

## Sample of image deskewing using "https://github.com/sbrunner/deskew"

In [5]:
#replace this with your own files, and paths.
sample_img = out_path_myhand+"/Original_img/delicious.jpg"

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [6]:
outputimage = Image.open(sample_img)
print(outputimage)
outputimage


NameError: name 'sample_img' is not defined

In [14]:
image = ioski.imread(sample_img)
grayscale = rgb2gray(image)
angle = determine_skew(grayscale)
print(angle)
rotated = rotate(image, angle, resize=True) * 255
print(type(rotated))
ioski.imsave("output_deskewed.png", rotated.astype(np.uint8))

-10.055865921787714
<class 'numpy.ndarray'>


In [7]:
Image.open("output_deskewed.png")

FileNotFoundError: [Errno 2] No such file or directory: 'output_deskewed.png'

In [16]:
def pre_process_image(image):
    """This function will pre-process a image with: cv2 & deskew
    so it can be process by tesseract"""
    img = cv2.imread(image)
    img = cv2.resize(img, None, fx=.3, fy=.3) #resize using percentage
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #change color format from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #format image to gray scale
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 11) #to remove background
    return img


In [17]:
def img_process(image):
    """This function will deskew"""
    cv2output = io.imread(image)
    angle = determine_skew(cv2output)
    print("angle rotation of image",angle)
    rotated = rotate(cv2output, angle, resize=True) * 255
    return rotated



In [18]:
#use function pre_process_image to output processed image to a saved file.
processed_img = pre_process_image("output_deskewed.png")
cv2.imwrite("output_processed.png",processed_img)
processed_img = pre_process_image(sample_img)
cv2.imwrite("output_no_deskewing.png",processed_img)


True

In [19]:
ground_truth = "Delicious substitute for tuna in your favorite recipes. Perfect for creative hors d'oeuvres and snacks. Great as a salad topper"

In [21]:
#pytesseract no preprocessing
hypothesis0 = pytesseract.image_to_string(sample_img)
print("CER, WER, Rate :",cer(ground_truth, hypothesis0),wer(ground_truth, hypothesis0),fuzz.ratio(ground_truth, hypothesis0))
print(hypothesis0)

CER, WER, Rate : 100.0 100.0 0



In [22]:
#pytesseract out of the box with deskew & rgb2gray
hypothesis1 = pytesseract.image_to_string("output_deskewed.png")
print("CER, WER, Rate :",cer(ground_truth, hypothesis1),wer(ground_truth, hypothesis1),fuzz.ratio(ground_truth, hypothesis1))
print(hypothesis1)

CER, WER, Rate : 46.82539682539682 147.61904761904762 56
      
  

   

IMO DS Sobs f; tute £o/ fu %e.. IY] youl Favorite
ec pes, 7 A kgec t Fos Crluti Ve hors d’ocuvres and.
Boe eC: er hat oa Ea lad topper.



In [23]:
#pytesseract out of the box with preprocessing (no deskewing)
hypothesis2 = pytesseract.image_to_string("output_no_deskewing.png")
print("CER, WER, Rate :",cer(ground_truth, hypothesis2),wer(ground_truth, hypothesis2),fuzz.ratio(ground_truth, hypothesis2))
print(hypothesis2)

CER, WER, Rate : 76.98412698412699 100.0 27
>. tune on r Favoikt-
re 3 yee 2



In [24]:
#pytesseract out of the box with preprocessing w/ deskewing
hypothesis3 = pytesseract.image_to_string("output_processed.png")
print("CER, WER, Rate :",cer(ground_truth, hypothesis3),wer(ground_truth, hypothesis3),fuzz.ratio(ground_truth, hypothesis3))
print(hypothesis3)

CER, WER, Rate : 19.047619047619047 66.66666666666667 74
       
 

Delicious substi tote for tune. In your Favorite
pec pes. eptect For Creative hors diocuyres and
Cote CAS: Great as & Sa. lod topper.

~



In [25]:
#pytesseract with ssq.traineddata, preprocessing
hypothesis4 = pytesseract.image_to_string("output_processed.png", lang="ssq")
print("CER, WER, Rate :",cer(ground_truth, hypothesis4),wer(ground_truth, hypothesis4),fuzz.ratio(ground_truth, hypothesis4))
print(hypothesis4)

CER, WER, Rate : 26.19047619047619 85.71428571428571 71
       
 

WDelicious sobsti tute for tuo. im your Fevouitc
*rec ipcs. oncect for arcetive hors docuyres amd
Coze.ckhS. Great asda Selod toppe..



In [26]:
#pytesseract with eng_ju.traineddata on trained data with preprocessing
hypothesis_Trained = pytesseract.image_to_string(out_path_myhand+"/training_dir_img/eng.juhand.exp13.png", lang="engju",config="--psm 6")
ground_truth_1 = 'A BCDE FGHIJKLMNOPORSTUVWXYZ'
print("CER, WER, Rate :",cer(ground_truth_1, hypothesis_Trained),wer(ground_truth_1, hypothesis_Trained),fuzz.ratio(ground_truth_1, hypothesis_Trained))
print(hypothesis_Trained)

CER, WER, Rate : 0.0 0.0 97
A BCDE FGHIJKLMNOPORSTUVWXYZ



In [27]:
#pass the image with text(handwritten) to google vision output string
with io.open("output_processed.png",'rb') as image_file:
    content = image_file.read()

image_v = vision.types.Image(content=content)
response = client.document_text_detection(image=image_v)
docText = response.full_text_annotation.text
hypothesis5 = docText

print("CER, WER, Rate :",cer(ground_truth, hypothesis5),wer(ground_truth, hypothesis5),fuzz.ratio(ground_truth, hypothesis5))
print(hypothesis5)

CER, WER, Rate : 16.666666666666668 14.285714285714286 88
Favorite
Delicious substitute for furnia in
your
recipes. Perfect for creative hors d'oeuvres and
Snacks. Great as a salad topper.



In [28]:
#pass the image with text(handwritten) to google vision output string
with io.open("output_deskewed.png",'rb') as image_file:
    content = image_file.read()

image_v = vision.types.Image(content=content)
response = client.document_text_detection(image=image_v)
docText = response.full_text_annotation.text
hypothesis6 = docText
print("CER, WER, Rate :",cer(ground_truth, hypothesis6),wer(ground_truth, hypothesis6),fuzz.ratio(ground_truth, hypothesis6))
print(hypothesis6)

CER, WER, Rate : 0.7936507936507936 4.761904761904762 95
Delicious substitute for tune in your Favorite
recipes. Perfect for creative hors d'oeuvres and
Snacks. Great as a salad topper.



In [29]:
print(f"Ground Truth text:\n\n{ground_truth}\n\n Extracted Text by Google Vision:\n\n{hypothesis6}")

Ground Truth text:

Delicious substitute for tuna in your favorite recipes. Perfect for creative hors d'oeuvres and snacks. Great as a salad topper

 Extracted Text by Google Vision:

Delicious substitute for tune in your Favorite
recipes. Perfect for creative hors d'oeuvres and
Snacks. Great as a salad topper.



In [30]:
print("Google Vision")
print("""Compares a ground truth and transcript file\nand calculates word error rate (WER)
between these two files\n""",wer(ground_truth, hypothesis5))
print("Number of Words",len(ground_truth.split()))
print("\n")
print("""Compares a ground truth and transcript file\nand calculates character error rate (CER)
between these two files\n""",cer(ground_truth, hypothesis5))
print("""\nIn information theory, linguistics, and computer science,\nthe Levenshtein distance is a string metric for measuring the difference between two sequences.\nInformally, the Levenshtein distance between two words is the minimum number of single-character \nedits (insertions, deletions or substitutions) required to change one word into the other.\nIt is named after the Soviet mathematician Vladimir Levenshtein, who considered this distance in 1965.\nwhere we can have a score out of 100,\nthat denotes two string are equal by giving similarity index.""")
print("Rate",fuzz.ratio(ground_truth, hypothesis5))
print("\nNumber of characters",len(ground_truth.replace(" ", "")))



Google Vision
Compares a ground truth and transcript file
and calculates word error rate (WER)
between these two files
 14.285714285714286
Number of Words 20


Compares a ground truth and transcript file
and calculates character error rate (CER)
between these two files
 16.666666666666668

In information theory, linguistics, and computer science,
the Levenshtein distance is a string metric for measuring the difference between two sequences.
Informally, the Levenshtein distance between two words is the minimum number of single-character 
edits (insertions, deletions or substitutions) required to change one word into the other.
It is named after the Soviet mathematician Vladimir Levenshtein, who considered this distance in 1965.
where we can have a score out of 100,
that denotes two string are equal by giving similarity index.
Rate 88

Number of characters 108


In [31]:
print("pytesseract with ssq.traineddata, preprocessing")
print("""Compares a ground truth and transcript file\nand calculates word error rate (WER)
between these two files\n""",wer(ground_truth, hypothesis3))
print("Number of Words",len(ground_truth.split()))
print("\n")
print("""Compares a ground truth and transcript file\nand calculates character error rate (CER)
between these two files\n""",cer(ground_truth, hypothesis3))
print("""\nIn information theory, linguistics, and computer science,\nthe Levenshtein distance is a string metric for measuring the difference between two sequences.\nInformally, the Levenshtein distance between two words is the minimum number of single-character \nedits (insertions, deletions or substitutions) required to change one word into the other.\nIt is named after the Soviet mathematician Vladimir Levenshtein, who considered this distance in 1965.\nwhere we can have a score out of 100,\nthat denotes two string are equal by giving similarity index.""")
print("Rate",fuzz.ratio(ground_truth, hypothesis3))
print("\nNumber of characters",len(ground_truth.replace(" ", "")))

pytesseract with ssq.traineddata, preprocessing
Compares a ground truth and transcript file
and calculates word error rate (WER)
between these two files
 66.66666666666667
Number of Words 20


Compares a ground truth and transcript file
and calculates character error rate (CER)
between these two files
 19.047619047619047

In information theory, linguistics, and computer science,
the Levenshtein distance is a string metric for measuring the difference between two sequences.
Informally, the Levenshtein distance between two words is the minimum number of single-character 
edits (insertions, deletions or substitutions) required to change one word into the other.
It is named after the Soviet mathematician Vladimir Levenshtein, who considered this distance in 1965.
where we can have a score out of 100,
that denotes two string are equal by giving similarity index.
Rate 74

Number of characters 108
